# Encontrar as raízes de uma função matemática de 2ª ordem

---

**Implementado com algoritmo de Holland**

---


Objetivo: Dados os coeficientes a, b e c de uma função  polinomial de segundo grau com raízes inteiras. Encontrar as raízes dessa função.

> *f(x): ax² + bx + c*


**Codificação do cromossomo:**

```
individuo = [[b11, b12, b13, b14, b15], [b21, b22, b23, b24, b25]]
```
*  As duas listas representam as duas raízes em binário.
*  O primeiro dígito de cada lista (b11 e b21) é designado ao sinal.

In [1]:
from random import randint

**População inicial**

In [2]:
# Gera aleatoriamente uma população de indivíduos
def inicia_populacao(tam):
    return [[[randint(0,1) for _ in range(5)], [randint(0,1) for _ in range(5)]] for _ in range(tam)]

**Cálculo de adaptação**

O cálculo de adaptação segue os passos:
* Converte os valores *x1* e *x2* armazenados nos cromossomos para decimal;
* Calcula a soma e o produto de *x1* e *x2*;
* Calcula a diferença absoluta entre a soma obtida e a soma exata, faz o mesmo para o produto;
* Multiplica as diferenças por -1. A soma dessas é a aptidão.

A aptidão de um indivíduo é um número *adap(individuo) ≤ 0*. De modo que,
* Se *adap(individuoᵢ) = 0*, *x1* e *x2* são raízes da função, e portanto, individuoᵢ é solução da função.
* Se *adap(individuoᵢ) < 0*, quanto menor esse número, mais distante está da raiz.

Deseja-se maximizar a adaptação, isto é, encontrar o indivíduo com adaptação nula.

In [3]:
# Recebe uma lista de 0 e 1 e retorna um decimal equivalente
def converte_decimal(binario):

    decimal = 0
    for i in range(4, 0, -1):
        decimal += binario[i] * 2 ** (4-i)

    return decimal if binario[0] == 0 else decimal * (-1)

In [4]:
def calcula_adaptacao(populacao, soma_raizes, produto_raizes):

    adaptacoes = []
    for individuo in populacao:
        x1 = converte_decimal(individuo[0])
        x2 = converte_decimal(individuo[1])

        soma, prod = x1 + x2, x1 * x2

        adap1 = abs(soma - soma_raizes)
        adap2 = abs(prod - produto_raizes)

        adaptacao = (-1 * adap1) + (-1 * adap2)
        adaptacoes.append(adaptacao)

    return adaptacoes

**Seleção dos mais adaptados**

In [5]:
# Ordena a população do mais adaptado ao menos adaptado
def ordena(pop, adap):

    tam = len(pop)
    for i in range(tam):
        for j in range(tam-1, i, -1):
            if adap[i] < adap[j]:
                adap[i], adap[j] = adap[j], adap[i]
                pop[i], pop[j] = pop[j], pop[i]

In [6]:
# Escolhe os 5 indivíduos mais adaptados da população
def seleciona(populacao, adaptacoes):

    ordena(populacao, adaptacoes)
    del populacao[5:]

**Cruzamento**

*cruza* gera filhos a partir da combinação de dois indivíduos (cromossomos) após a seleção dos mais adaptados. Essa função é aplicada em todos os pares da população.

In [7]:
def cruza(populacao):

    filhos = []
    for ind1 in populacao:
        for ind2 in populacao:
            if ind1 != ind2:
                r1 = randint(1,5)
                r2 = randint(1,5)
                f1 = [ind1[0][:r1] + ind2[0][r1:], ind1[1][:r2] + ind2[1][r2:]]
                f2 = [ind2[0][:r1] + ind1[0][r1:], ind2[1][:r2] + ind1[1][r2:]]
                if f1 not in filhos and f1 not in populacao:
                    filhos.append(f1)
                if f2 not in filhos and f2 not in populacao:
                    filhos.append(f2)

    populacao += filhos

**Mutação**

In [8]:
def muta(populacao):

    for individuo in populacao:
        r = randint(0,100)
        if r >= 90:
            for i in range(5):
                r = randint(0,1)
                if r == 1:
                    individuo[0][i] = int(not individuo[0][i])
                r = randint(0,1)
                if r == 1:
                    individuo[1][i] = int(not individuo[1][i])

---
**Execução**

In [9]:
def print_populacao(populacao, adaptacoes):

    for ind, adap in zip(populacao, adaptacoes):
        print(ind, 'Adap:', adap)
    print()

In [10]:
def encontra_raizes(a, b, c):

    # Gerar uma população inicial com 10 indivíduos
    populacao = inicia_populacao(10)
    soma = -1 * (b / a)  # Calcular a soma exata
    produto = c / a  # Calcular o produto exato
    # Verificar as adaptações dos indivíduos iniciais
    adaptacoes = calcula_adaptacao(populacao, soma, produto)
    print(f'População inicial:')
    print_populacao(populacao, adaptacoes)

    seleciona(populacao, adaptacoes)
    iteracao = 0

    while max(adaptacoes) < 0:
        cruza(populacao)
        muta(populacao)
        adaptacoes = calcula_adaptacao(populacao, soma, produto)
        seleciona(populacao, adaptacoes)
        iteracao += 1
        print(f'População pós iteração {iteracao}:')
        print_populacao(populacao, adaptacoes)

        if iteracao == 100:
            print('\nSOLUÇÃO NÃO ENCONTRADA!\n')
            return None

    return populacao[0]

**Função 1:**

𝓕(𝓧) = 𝓧² - 5𝓧 - 36

In [11]:
a, b, c = 1, -5, -36
solucao = encontra_raizes(a, b, c)
if solucao is not None:
    print(f'{solucao=}\n')
    x1, x2 = converte_decimal(solucao[0]), converte_decimal(solucao[1])
    print(f'{x1} e {x2} são raízes da função f(x): {a}x² + {b}x + {c}')

População inicial:
[[1, 1, 0, 0, 0], [0, 1, 0, 0, 0]] Adap: -33.0
[[0, 1, 0, 1, 1], [1, 0, 0, 1, 0]] Adap: -18.0
[[0, 0, 1, 0, 0], [0, 1, 1, 0, 1]] Adap: -100.0
[[1, 1, 0, 1, 1], [0, 1, 1, 1, 0]] Adap: -120.0
[[0, 0, 0, 0, 1], [0, 0, 0, 0, 0]] Adap: -40.0
[[1, 1, 0, 0, 0], [0, 0, 0, 0, 0]] Adap: -49.0
[[0, 0, 0, 1, 0], [1, 0, 1, 1, 0]] Adap: -33.0
[[0, 1, 0, 1, 0], [1, 1, 0, 0, 1]] Adap: -58.0
[[1, 1, 0, 0, 0], [0, 1, 1, 0, 0]] Adap: -61.0
[[0, 1, 1, 1, 1], [0, 1, 1, 0, 1]] Adap: -254.0

População pós iteração 1:
[[0, 1, 0, 1, 1], [1, 0, 0, 1, 0]] Adap: -18.0
[[0, 1, 0, 1, 0], [1, 0, 0, 1, 0]] Adap: -19.0
[[0, 1, 0, 0, 0], [1, 0, 0, 1, 0]] Adap: -21.0
[[0, 0, 1, 0, 1], [1, 0, 0, 1, 1]] Adap: -24.0
[[0, 1, 0, 1, 0], [1, 0, 1, 1, 0]] Adap: -25.0

População pós iteração 2:
[[0, 1, 0, 1, 1], [1, 0, 0, 1, 1]] Adap: -6.0
[[0, 1, 1, 0, 1], [1, 0, 0, 1, 1]] Adap: -8.0
[[0, 1, 0, 1, 0], [1, 0, 0, 1, 1]] Adap: -8.0
[[0, 0, 1, 0, 1], [1, 0, 1, 1, 0]] Adap: -12.0
[[0, 1, 0, 0, 0], [1, 0, 1, 1, 0]]

**Função 2:**

𝓕(𝓧) = 2𝓧² - 8𝓧 + 8

In [12]:
a, b, c = 2, -8, 8
solucao = encontra_raizes(a, b, c)
if solucao is not None:
    print(f'{solucao=}\n')
    x1, x2 = converte_decimal(solucao[0]), converte_decimal(solucao[1])
    print(f'{x1} e {x2} são raízes da função f(x): {a}x² + {b}x + {c}')

População inicial:
[[1, 1, 1, 0, 1], [0, 0, 1, 0, 0]] Adap: -69.0
[[0, 0, 1, 1, 0], [1, 1, 0, 1, 0]] Adap: -72.0
[[1, 1, 1, 0, 1], [1, 0, 0, 0, 1]] Adap: -27.0
[[1, 0, 0, 0, 1], [1, 1, 1, 0, 0]] Adap: -25.0
[[1, 1, 1, 0, 0], [1, 0, 1, 0, 0]] Adap: -64.0
[[0, 1, 1, 0, 1], [0, 0, 1, 1, 1]] Adap: -103.0
[[0, 0, 1, 0, 0], [0, 1, 1, 1, 0]] Adap: -66.0
[[0, 1, 0, 1, 0], [1, 0, 0, 1, 1]] Adap: -37.0
[[0, 1, 1, 0, 1], [1, 0, 0, 1, 0]] Adap: -37.0
[[1, 0, 1, 0, 0], [0, 0, 0, 0, 0]] Adap: -12.0

População pós iteração 1:
[[1, 0, 0, 0, 0], [1, 0, 0, 0, 0]] Adap: -8.0
[[1, 0, 0, 0, 1], [1, 0, 0, 1, 0]] Adap: -9.0
[[1, 0, 0, 0, 1], [1, 0, 0, 0, 1]] Adap: -9.0
[[1, 0, 0, 0, 1], [1, 0, 0, 0, 0]] Adap: -9.0
[[0, 1, 0, 1, 0], [1, 0, 0, 0, 0]] Adap: -10.0

População pós iteração 2:
[[1, 0, 0, 0, 0], [1, 0, 0, 0, 0]] Adap: -8.0
[[1, 0, 0, 1, 0], [1, 0, 0, 1, 0]] Adap: -8.0
[[0, 1, 0, 0, 0], [1, 0, 0, 0, 0]] Adap: -8.0
[[1, 0, 0, 0, 1], [1, 0, 0, 0, 0]] Adap: -9.0
[[1, 0, 0, 0, 1], [1, 0, 0, 1, 0]] Adap: 

**Função 3:**

𝓕(𝓧) = 2𝓧² - 8𝓧 - 8

In [13]:
a, b, c = 2, -8, -8
solucao = encontra_raizes(a, b, c)
if solucao is not None:
    print(f'{solucao=}\n')
    x1, x2 = converte_decimal(solucao[0]), converte_decimal(solucao[1])
    print(f'{x1} e {x2} são raízes da função f(x): {a}x² + {b}x + {c}')

População inicial:
[[0, 1, 0, 1, 1], [0, 1, 0, 1, 1]] Adap: -143.0
[[1, 1, 0, 1, 1], [1, 0, 1, 0, 1]] Adap: -79.0
[[0, 1, 1, 1, 0], [1, 1, 0, 0, 0]] Adap: -110.0
[[1, 1, 1, 1, 1], [0, 0, 0, 0, 1]] Adap: -29.0
[[0, 1, 1, 0, 1], [0, 0, 0, 1, 0]] Adap: -41.0
[[0, 0, 0, 0, 0], [0, 0, 0, 1, 0]] Adap: -6.0
[[0, 0, 0, 1, 1], [1, 0, 0, 1, 0]] Adap: -5.0
[[0, 1, 0, 1, 0], [1, 1, 1, 0, 0]] Adap: -122.0
[[1, 0, 1, 1, 0], [1, 1, 0, 0, 0]] Adap: -70.0
[[1, 1, 0, 1, 1], [1, 0, 0, 0, 1]] Adap: -31.0

População pós iteração 1:
[[0, 0, 0, 1, 1], [1, 0, 0, 0, 1]] Adap: -3.0
[[0, 0, 0, 1, 0], [1, 0, 0, 1, 0]] Adap: -4.0
[[0, 0, 0, 1, 1], [1, 0, 0, 1, 0]] Adap: -5.0
[[0, 0, 0, 0, 0], [0, 0, 0, 1, 0]] Adap: -6.0
[[0, 0, 0, 0, 1], [0, 0, 0, 0, 1]] Adap: -7.0

População pós iteração 2:
[[0, 0, 0, 1, 1], [1, 0, 0, 0, 1]] Adap: -3.0
[[0, 0, 0, 1, 0], [1, 0, 0, 1, 0]] Adap: -4.0
[[0, 0, 0, 1, 1], [1, 0, 0, 1, 0]] Adap: -5.0
[[0, 0, 0, 1, 0], [1, 0, 0, 0, 1]] Adap: -5.0
[[0, 0, 0, 0, 0], [0, 0, 0, 1, 1]] Adap: -